In [12]:
import googleapiclient.discovery
from googleapiclient.errors import HttpError
from textblob import TextBlob
import matplotlib.pyplot as plt
import pandas as pd
from langdetect import detect, LangDetectException

In [2]:
user_agent = "Brand sentiment analysis"
reddit = praw.Reddit(
  client_id="ujQpmAa-rxhI6tgE246E4w",
  client_secret="BnhybdI_fallhH64Us6vutIfsXP21Q",
  user_agent=user_agent
)

In [4]:
def fetch_reddit_comments(subreddit, query, limit=100, months=6):
    subreddit = reddit.subreddit(subreddit)
    posts = []
    for post in subreddit.search(query, limit=limit):
        post_date = datetime.utcfromtimestamp(post.created_utc)
        post.comments.replace_more(limit=0)  # Ensure all comments are fetched
        comments = post.comments.list()
        #print(comments)
        if comments:
            for comment in comments:
                if hasattr(comment, 'body') and comment.body.strip():  # Ensure comment body is not empty or just whitespace
                    try:
                        comment_date = datetime.utcfromtimestamp(comment.created_utc)
                        if detect(comment.body) == 'en' and comment_date >= datetime.now() - timedelta(days=months*30):  # Adjust to the specific time frame
                            posts.append([comment_date, post.title, comment.body, post.url])
                            english = True
                    except LangDetectException:  # Handle cases where language detection fails
                        continue

    df = pd.DataFrame(posts, columns=["Date", "Title", "Comment", "Url"])
    df['Date'] = pd.to_datetime(df['Date'], unit='s')
    df['Source'] = 'Reddit'
    if english == True: 
        df['Language'] = 'English'
    return df

In [5]:
df_neutrogena = fetch_reddit_comments('all', 'neutrogena', 100)
df_neutrogena.to_csv('neutrogena.csv', sep=',', index=False, encoding='utf-8')

In [6]:
# Load sentiment analysis pipeline
#model_name = "distilbert-base-uncased-finetuned-sst-2-english" #only pos and neg
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_name)
sentiment_pipeline = pipeline("sentiment-analysis", model=model_name)

def truncate_text(text, max_tokens=510):
    tokens = tokenizer.tokenize(text)
    if len(tokens) > max_tokens:
        tokens = tokens[:max_tokens]
    return tokenizer.convert_tokens_to_string(tokens)

def get_sentiment_score(text):
    truncated_text = truncate_text(text)
    result = sentiment_pipeline(truncated_text)[0]
    return result['label'], result['score']

# Apply sentiment analysis
df_neutrogena['Sentiment'], df_neutrogena['Sentiment_Score'] = zip(*df_neutrogena['Comment'].apply(get_sentiment_score))

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
df_neutrogena

,Date,Title,Comment,Url,Source,Language,Sentiment,Sentiment_Score,Week
0,2024-01-19 18:50:34,[Misc] Your thoughts on Neutrogena?,"In my skincare routine, I'm unsure whether I ...",https://www.reddit.com/r/SkincareAddiction/com...,Reddit,English,neutral,0.909842,2024-01-15
1,2024-01-21 02:30:22,[Misc] Your thoughts on Neutrogena?,Neutrogena now marketing wrinkle cream to chil...,https://www.reddit.com/r/SkincareAddiction/com...,Reddit,English,neutral,0.704988,2024-01-15
2,2024-01-31 06:11:00,[Misc] Your thoughts on Neutrogena?,The orange oil free acne cleanser WOULDVE been...,https://www.reddit.com/r/SkincareAddiction/com...,Reddit,English,positive,0.718628,2024-01-29
3,2024-01-12 13:05:43,What are your thoughts on Neutrogena sunscreen ?,Is it necessary to double cleans for this suns...,https://i.redd.it/ultpbnpxf1ya1.jpg,Reddit,English,neutral,0.899456,2024-01-08
4,2024-04-06 03:25:24,What are your thoughts on Neutrogena sunscreen ?,How often do you reapply it??,https://i.redd.it/ultpbnpxf1ya1.jpg,Reddit,English,neutral,0.952640,2024-04-01
...,...,...,...,...,...,...,...,...,...
911,2024-01-02 02:54:33,Neutrogena hydroboost helpful?,I just would not do too much strong stuff. Loo...,https://www.reddit.com/r/Rosacea/comments/18vt...,Reddit,English,neutral,0.730100,2024-01-01
912,2024-06-17 10:57:47,Brand new Neutrogena holika for sale,"(Thank you for posting, please ensure you are ...",https://www.reddit.com/gallery/1dhvafa,Reddit,English,neutral,0.823992,2024-06-17
913,2024-06-03 05:34:15,Selling Neutrogena Sleeping Mask with Kult box,MRP : 1150. Its a kult freebie. Selling for 30...,https://i.redd.it/1jqv0szzla4d1.jpeg,Reddit,English,neutral,0.794685,2024-06-03
914,2024-06-03 05:32:09,Selling Neutrogena Sleeping Mask with Kult box,Thank you for your submission to r/IndianBeaut...,https://i.redd.it/1jqv0szzla4d1.jpeg,Reddit,English,neutral,0.710653,2024-06-03


# by week

In [17]:
# Convert the 'Date' column to datetime
df_neutrogena['Date'] = pd.to_datetime(df_neutrogena['Date'])

# Group by week and sentiment
df_neutrogena['Week'] = df_neutrogena['Date'].dt.to_period('W').apply(lambda r: r.start_time)
weekly_sentiment = df_neutrogena.groupby(['Week', 'Sentiment']).size().unstack(fill_value=0)

# Create the plot
fig = go.Figure()

# Add positive sentiments
if 'positive' in weekly_sentiment:
    fig.add_trace(go.Bar(
        x=weekly_sentiment.index,
        y=weekly_sentiment['positive'],
        name='Positive',
        marker_color='green'
    ))

# Add neutral sentiments
if 'neutral' in weekly_sentiment:
    fig.add_trace(go.Bar(
        x=weekly_sentiment.index,
        y=weekly_sentiment['neutral'],
        name='Neutral',
        marker_color='orange'
    ))

# Add negative sentiments (inverted to point downwards)
if 'negative' in weekly_sentiment:
    fig.add_trace(go.Bar(
        x=weekly_sentiment.index,
        y=-weekly_sentiment['negative'],
        name='Negative',
        marker_color='red'
    ))

# Update layout
fig.update_layout(
    title='Sentiment Over Time',
    xaxis_title='Time',
    yaxis_title='Count',
    barmode='group',
    bargap=0.2,
    bargroupgap=0.1,
    legend_title_text='Sentiment'
)

# Show the plot
fig.show()

# by days

In [18]:
# Group by day and sentiment
df_neutrogena['Day'] = df_neutrogena['Date'].dt.to_period('D').apply(lambda r: r.start_time)
daily_sentiment = df_neutrogena.groupby(['Day', 'Sentiment']).size().unstack(fill_value=0)

# Create the plot
fig = go.Figure()

# Add positive sentiments
if 'positive' in daily_sentiment:
    fig.add_trace(go.Bar(
        x=daily_sentiment.index,
        y=daily_sentiment['positive'],
        name='Positive',
        marker_color='green'
    ))

# Add neutral sentiments
if 'neutral' in daily_sentiment:
    fig.add_trace(go.Bar(
        x=daily_sentiment.index,
        y=daily_sentiment['neutral'],
        name='Neutral',
        marker_color='orange'
    ))

# Add negative sentiments (inverted to point downwards)
if 'negative' in daily_sentiment:
    fig.add_trace(go.Bar(
        x=daily_sentiment.index,
        y=-daily_sentiment['negative'],
        name='Negative',
        marker_color='red'
    ))

# Update layout
fig.update_layout(
    title='Daily Sentiment Over Time',
    xaxis_title='Time',
    yaxis_title='Count',
    barmode='group',  # Change to group bars side by side
    bargap=0.2,
    bargroupgap=0.1,
    legend_title_text='Sentiment',
)

# Show the plot
fig.show()